In [1]:
from typing import List, Optional, Generator
import typing
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

Atcoder Beginner contest: 365

In [4]:
def leap_year(year: int) -> int:
    if year % 4 != 0:
        return 365
    elif year % 100 != 0:
        return 366
    elif year % 400 != 0:
        return 365
    else:
        return 366

# Testing the solution
assert leap_year(2023) == 365
assert leap_year(1992) == 366
assert leap_year(1800) == 365
assert leap_year(1600) == 366

# %$
def second_largest(N: int, nums: list[int]) -> int:
    max_ = max(nums)
    idx = 0 if nums[0] != max_ else 1
    for i in range(N):
        if nums[idx] < nums[i] < max_:
            idx = i
    return idx + 1

# Testing the solution
assert second_largest(5, [3,8,2,5,1]) == 4
assert second_largest(4, [8,2,5,1]) == 3
assert second_largest(2, [2,1]) == 2
assert second_largest(8, [1,2,3,4,5,10,9,11]) == 6

In [5]:
def transportation_expenses(N: int, M: int, costs: list[int]) -> int:
    total_cost = sum(costs)
    if total_cost <= M:
        return -1
    else:
        low, high = 0, M
        while low <= high:
            mid = (low + high) // 2
            total_cost = sum(map(lambda x: min(x, mid), costs))
            if total_cost <= M:
                low = mid + 1
            else:
                high = mid - 1

        return high

# Testing the solution
assert transportation_expenses(4, 8, [1,3,2,4]) == 2
assert transportation_expenses(3, 20, [5,3,2]) == -1
assert transportation_expenses(10, 23, [2,5,6,5,2,1,7,9,7,2]) == 2

In [6]:
def RPS(N: int, moves: str) -> int:

    win_mapping: dict[str, str] = {"R": "P", "S": "R", "P": "S"}
    move_mapping: dict[int, str] = dict(enumerate("RPS"))
    move_mapping_rev: dict[str, int] = {j: i for i, j in move_mapping.items()}

    dp: list[int] = [0 for j in range(3)]
    for i in range(N - 1, -1, -1):
        curr, win_move = moves[i], win_mapping[moves[i]]
        next_dp: list[int] = [0 for i in range(3)]
        for j in range(3):
            prev = move_mapping[j]
            if win_move != prev:
                next_dp[j] = max(next_dp[j], 1 + dp[move_mapping_rev[win_move]])
            if moves[i] != prev:
                next_dp[j] = max(next_dp[j], dp[move_mapping_rev[curr]])
        dp = next_dp

    return max(dp)

# Testing the solution
assert RPS(24, "SPRPSRRRRRPPRPRPSSRSPRSS") == 18
assert RPS(10, "SSSSSSSSSS") == 5
assert RPS(5, "SSPRS") == 4
assert RPS(6, "PRSSRS") == 5

#### Leetcode Biweekly

In [7]:
def minFlips(grid: list[list[int]]) -> int:
    M, N = len(grid), len(grid[0])

    # Count flips required horizontally
    h_flips = 0
    for i in range(M):
        j, k, flips = 0, N - 1, 0
        while j < k:
            if grid[i][j] != grid[i][k]:
                flips += 1
            j, k = j + 1, k - 1
        h_flips += flips

    # Count flips required vertically
    v_flips = 0
    for i in range(N):
        j, k, flips = 0, M - 1, 0
        while j < k:
            if grid[j][i] != grid[k][i]:
                flips += 1
            j, k = j + 1, k - 1
        v_flips += flips

    return min(h_flips, v_flips)

# Testing the solution
assert minFlips([[1,0,0],[0,0,0],[0,0,1]]) == 2
assert minFlips([[0,1],[0,1],[0,0]]) == 1
assert minFlips([[1],[0]]) == 0
assert minFlips([[1]]) == 0

#### Upsolving Min Flips 2

In [8]:
def minFlipsII(grid: list[list[int]]) -> int:
    # https://leetcode.com/problems/minimum-number-of-flips-to-make-binary-grid-palindromic-ii/solutions/5581114/counting-greedy-python3/
    def symmetric_pairs(i: int, j: int) -> set[tuple[int, int]]:
        return {(i, j), (i, N - j - 1), (M - i - 1, j), (M - i - 1, N - j - 1)}

    # Compute dimensions
    M, N = len(grid), len(grid[0])

    # Calculate the number of flips
    # 4 way symmetry - all 0s or all 1s
    # 2 way symmetry - based on odd / even count of (1, 1), (0, 1) / (1, 0) we decide
    # 1 cell symmetry - all 0s

    flips, tws10, tws11 = 0, 0, 0
    visited: set[tuple[int, int]] = set()
    for i in range(M):
        for j in range(N):
            if (i, j) not in visited:
                ones, pairs = 0, symmetric_pairs(i, j)
                for x, y in pairs:
                    if grid[x][y] == 1:
                        ones += 1
                    visited.add((x, y))

                # Four cell symmetry
                if len(pairs) == 4:
                    flips += min(ones, len(pairs) - ones)

                # Single cell symmetry
                elif len(pairs) == 1:
                    flips += ones

                # Two cell symmetry
                else:
                    # Count 1, 1
                    if ones == 2:
                        tws11 += 1
                    # Count 1, 0 / 0, 1
                    elif ones == 1:
                        tws10 += 1

    # If even number of 1, 1 pairs
    if tws11 % 2 == 0:
        # Change all (0, 1), (1, 0) to (0, 0)
        flips += tws10
    else:
        # If there is atleast a single (1, 0) pair change all (1, 0) that to (1, 1)
        # Otherwise change a single (1, 1) to (0, 0)
        flips += tws10 if tws10 else 2

    return flips

# Testing the solution
assert minFlipsII([[1,0,0],[0,1,0],[0,0,1]]) == 3
assert minFlipsII([[0,1],[0,1],[0,0]]) == 2
assert minFlipsII([[1],[1]]) == 2
assert minFlipsII([[1],[1],[1],[0]]) == 1
assert minFlipsII([[1,1,0,0,1],[0,1,1,0,1],[0,0,1,1,1],[1,0,1,1,0],[1,0,1,1,0],[0,0,1,1,1],[0,1,1,0,1],[1,1,0,1,0]]) == 16

LC Weekly - 4th Aug 2024

In [9]:
class neighborSum:

    def __init__(self, grid: List[List[int]]):
        self.grid: list[list[int]] = grid
        self.M, self.N = len(grid), len(grid[0])
        self.positions: dict[int, tuple[int, int]] = {grid[i][j]: (i, j) for i in range(self.M) for j in range(self.N)}

    def adjacentSum(self, value: int) -> int:
        result = 0
        i, j = self.positions[value]
        for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
            if 0 <= x < self.M and 0 <= y < self.N:
                result += self.grid[x][y]
        return result

    def diagonalSum(self, value: int) -> int:
        result = 0
        i, j = self.positions[value]
        for x, y in ((i - 1, j - 1), (i + 1, j + 1), (i - 1, j + 1), (i + 1, j - 1)):
            if 0 <= x < self.M and 0 <= y < self.N:
                result += self.grid[x][y]
        return result

# Testing the solution
ns = neighborSum([[0,1,2], [3,4,5], [6,7,8]])
assert ns.adjacentSum(1) == 6
assert ns.adjacentSum(4) == 16
assert ns.diagonalSum(4) == 16
assert ns.diagonalSum(8) == 4

In [10]:
def shortestDistanceAfterQueries(N: int, queries: list[list[int]]) -> list[int]:
    def shortest_path() -> int:
        queue: collections.deque[tuple[int, int]] = collections.deque([(0, 0)])
        while queue:
            curr, dist = queue.popleft()
            for next_node in adjl[curr]:
                if next_node == N - 1:
                    return dist + 1
                else:
                    queue.append((next_node, dist + 1))

        return -1

    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for i in range(N):
        adjl[i].append(i + 1)

    result: list[int] = []
    for n1, n2 in queries:
        adjl[n1].append(n2)
        result.append(shortest_path())

    return result

# Testing the solution
assert shortestDistanceAfterQueries(5, [[2,4],[0,2],[0,4]]) == [3,2,1]
assert shortestDistanceAfterQueries(4, [[0,3],[0,2]]) == [1,1]

#### Matrix factorization GYM Practice: https://codeforces.com/gym/102644

In [11]:
def count_paths_brute(N: int, E: int, K: int, edges: list[tuple[int, int]]) -> int:
    # Convert edges into adjl
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n2].append(n1)

    # Do a 'cached' DFS
    MOD = 10 ** 9 + 7

    # Do a DFS traversal and cache the results
    dp: dict[tuple[int, int], int] = {(0, K + 1): 0}
    stack: list[tuple[int, int, int]] = [(i, 0, K) for i in range(1, N + 1)]
    while stack:
        curr, prev, k = stack[-1]
        if (curr, k) in dp or k == 0:
            if k == 0:
                dp[(curr, k)] = 1
            dp[(prev, k + 1)] = (dp[(prev, k + 1)] + dp[(curr, k)]) % MOD
            stack.pop()
        else:
            dp[(curr, k)] = 0
            for next_node in adjl[curr]:
                stack.append((next_node, curr, k - 1))

    return dp[(0, K + 1)]

# Testing the solution
assert count_paths_brute(3, 4, 2, [(1,2), (2,3), (3,1), (2,1)]) == 5
assert count_paths_brute(5, 10, 11, [(2,3), (4,2), (2,1), (2,4), (1,5), (5,2), (3,2), (3,1), (3,4), (1,2)]) == 21305

In [12]:
def count_paths(n_vertices: int, n_edges: int, K: int, edges: list[tuple[int, int]]) -> int:
    """
    Adj matrix raised to power K

    Each cell (i, j) denotes how many paths exist from node i + 1 to j + 1.
    Total paths would be the sum of all cells
    """

    def matrix_multiply(A: list[list[int]], B: list[list[int]]) -> list[list[int]]:
        M, N, P, Q = len(A), len(A[0]), len(B), len(B[0])
        assert N == P, "invalid matrix dimensions"
        result: list[list[int]] = [[0 for j in range(M)] for i in range(Q)]
        for i in range(M):
            for j in range(Q):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % MOD

        return result

    def matrix_power(base: list[list[int]], exp: int) -> list[list[int]]:
        N = len(base)
        result: list[list[int]] = [[1 if i == j else 0 for j in range(N)] for i in range(N)]
        while exp:
            if exp & 1:
                result = matrix_multiply(result, base)
            base = matrix_multiply(base, base)
            exp >>= 1

        return result

    # Return path count % MOD
    MOD = 10 ** 9 + 7
    adjm: list[list[int]] = [[0 for j in range(n_vertices)] for i in range(n_vertices)]
    for n1, n2 in edges:
        adjm[n1 - 1][n2 - 1] = 1

    # Raise Adjm to the power of K using matrix factorization
    result = matrix_power(adjm, K)
    return sum(sum(row) for row in result) % MOD

# Testing the solution
assert count_paths(3, 4, 2, [(1,2), (2,3), (3,1), (2,1)]) == 5
assert count_paths(5, 10, 11, [(2,3), (4,2), (2,1), (2,4), (1,5), (5,2), (3,2), (3,1), (3,4), (1,2)]) == 21305

In [13]:
def knight_paths(K: int) -> int:

    ij2n: typing.Callable[[int, int], int] = lambda x, y: 8 * x + y
    n2ij: typing.Callable[[int], tuple[int, int]] = lambda n: (n // 8, n % 8)

    def matrix_multiply(A: list[list[int]], B: list[list[int]], mod: int) -> list[list[int]]:
        M, N, P, Q = len(A), len(A[0]), len(B), len(B[0])
        assert N == P, "invalid dimensions"
        result: list[list[int]] = [[0 for j in range(Q)] for i in range(M)]
        for i in range(M):
            for j in range(Q):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % mod

        return result

    def matrix_power(base: list[list[int]], power: int, mod: int) -> list[list[int]]:
        N = len(base)
        result: list[list[int]] = [[1 if i == j else 0 for j in range(N)] for i in range(N)]
        while power:
            if power & 1:
                result = matrix_multiply(result, base, mod)
            base = matrix_multiply(base, base, mod)
            power //= 2

        return result

    def mark_paths(n: int) -> list[int]:
        x1, y1 = n2ij(n)
        result: list[int] = []
        for x_delta in (-2, -1, 1, 2):
            for y_delta in (-2, -1, 1, 2):
                x2, y2 = x1 + x_delta, y1 + y_delta
                if 0 <= x2 < 8 and 0 <= y2 < 8 and abs(x_delta) != abs(y_delta):
                    result.append(ij2n(x2, y2))

        return result

    @functools.cache
    def generate_base() -> list[list[int]]:
        # Count of paths the knight can make starting at given point
        base: list[list[int]] = [[0 for j in range(65)] for i in range(65)]
        for i in range(64):
            for j in mark_paths(i):
                base[i][j] = 1

        # We store the cells from 0 - 63, 64 stores the answer
        for i in range(65):
            base[i][64] = 1

        return base

    base = generate_base()
    result = matrix_power(base, K + 1, 4294967296)
    return result[0][64]

# Testing the solution
assert knight_paths(1) == 3
assert knight_paths(2) == 15
assert knight_paths(6) == 17231

#### Codeforces Div 2 - 4th July 2024

In [ ]:
def question_marks(N: int, answers: str) -> int:
    marks = 0
    correct: dict[str, int] = {ch: N for ch in "ABCD"}
    for answer in answers:
        if answer in correct:
            marks += 1
            correct[answer] -= 1
            if correct[answer] == 0:
                correct.pop(answer)

    return marks

# Testing the solution
assert question_marks(1, "ABCD") == 4
assert question_marks(2, "AAAAAAAA") == 2
assert question_marks(2, "AAAABBBB") == 4
assert question_marks(2, "????????") == 0
assert question_marks(3, "ABCABCABCABC") == 9
assert question_marks(5, "ACADC??ACAC?DCAABC?C") == 13

In [ ]:
def parity_and_sum(N: int, nums: list[int]) -> int:
    # Split into odd / even
    odd: list[int] = []
    even: list[int] = []
    for n in nums:
        if n % 2 == 0:
            even.append(n)
        else:
            odd.append(n)

    # Sort odd and even
    odd.sort(reverse=False)
    even.sort(reverse=True)

    # Check the smallest in odd and largest in even
    moves = 0
    while odd and even:
        moves += 1
        if even[-1] < odd[-1]:
            odd.append(even.pop() + odd[-1])
        else:
            odd[-1] += even[0]

    return moves

# Testing the solution
assert parity_and_sum(4, [1,4,10,12]) == 4
assert parity_and_sum(5, [1,3,5,7,9]) == 0
assert parity_and_sum(4, [4,4,4,4]) == 0
assert parity_and_sum(3, [2,3,4]) == 2
assert parity_and_sum(4, [3,2,2,8]) == 4
assert parity_and_sum(6, [4,3,6,1,2,1]) == 3

In [ ]:
def light_switches(N: int, K: int, install_time: list[int]) -> int:
    start, end = install_time[0], install_time[0] + K - 1
    for time in install_time[1:]:
        curr_start, curr_end = time, time + K - 1

        # Always ensure that start, end is the greater
        if start < curr_start:
            start, end, curr_start, curr_end = curr_start, curr_end, start, end

        # If they don't overlap, try to increase them to the nearest range
        if curr_end < start:
            factor = math.ceil((start - curr_end) / (2 * K))
            curr_start, curr_end = curr_start + (factor * 2 * K), curr_end + (factor * 2 * K)

        # Check if overlap
        start, end = max(start, curr_start), min(end, curr_end)

        if end < start:
            return -1

    return start

# Testing the solution
assert light_switches(3, 3, [6,2,1]) == 8
assert light_switches(7, 5, [14,34,6,25,46,7,17]) == 47
assert light_switches(4, 3, [2,3,4,5]) == -1
assert light_switches(4, 4, [2,3,4,5]) == 5
assert light_switches(4, 3, [3,4,8,9]) == 10
assert light_switches(1, 1, [1]) == 1
assert light_switches(6, 5, [40,80,99,60,90,50]) == 100
assert light_switches(6, 5, [64,40,50,68,70,10]) == -1